In [1]:
#!pip install langchain
#!pip install pypdf
#!pip install python-dotenv
#!pip install pinecone-client
#!pip install tiktoken
import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from dotenv import load_dotenv
import os
from openai import OpenAI
client = OpenAI(api_key="sk-")
load_dotenv()


True

<h1>Mining metadata</h1>

In [21]:
#Method for parsing metadata from LLM answer
def parse_metadata(input_string):
    parsed_data = {
        "title": "",
        "authors": "",
        "date": None
    }
    
    try:
        parts = input_string.strip().split("\n")  
        
        for part in parts:
            # Split key and value on ": "
            key, value = part.split(": ", 1)

            if "title" in key:
                parsed_data["title"] = value
            elif "authors" in key:
                parsed_data["authors"] = value
            elif "date" in key:
                # Attempting to parse the publication date
                parsed_data["date"] = value
            
    except ValueError:
        pass
    print(parsed_data)
    return parsed_data

In [22]:
#Method for prompting for metdata
def prompt_for_metadata(page_content):
    completion = client.chat.completions.create(
      model="gpt-4-turbo-preview",
      messages=[
        {"role": "system", "content": "You are a research librarian."},
        {"role": "user", "content": f"Given the context below, tell me what the title, authors and date of publication of the document is: document: {page_content} Answer in the format: title: answer \n authors: answer \n date: answer(YYYY)"}
      ]
    )
    print(completion.choices[0].message.content)
    metadata = parse_metadata(completion.choices[0].message.content)
    return metadata

<h1>Load data and mine metadata</h1>



In [24]:
# This loop wil load all data from files and mine metadata with llm
data = []
file_path = "Algae-Chatbot/"
file_names = os.listdir(file_path)

for file_name in file_names:
    if file_name.endswith('.pdf'):
        try:
            print(file_name)
            loader = PyPDFLoader(file_path + file_name)
            pages = loader.load_and_split()
            text_splitter = CharacterTextSplitter(chunk_size=2056, chunk_overlap=128)
            docs = text_splitter.split_documents(pages)
            metadata = prompt_for_metadata(docs[0].page_content)
            for doc in docs:
                doc.metadata['authors'] = metadata['authors']
                doc.metadata['publication_date'] = metadata['date']
                doc.metadata['title'] = metadata['title']
                doc.metadata['file_name'] = file_path + file_name
            data.append(docs)
            print("")
            print("")
        except Exception as e:
            print(f"An error occurred with {file_name}: {e}")




1-s2.0-S0196890409000764-main.pdf
title: Review on biofuel oil and gas production processes from microalgae
authors: Sarmidi Amin
date: 2009
{'title': 'Review on biofuel oil and gas production processes from microalgae', 'authors': 'Sarmidi Amin', 'date': '2009'}


1-s2.0-S0196890410002207-main.pdf
title: Use of algae as biofuel sources
authors: Ayhan Demirbas
date: 2010
{'title': 'Use of algae as biofuel sources', 'authors': 'Ayhan Demirbas', 'date': '2010'}


1-s2.0-S0360319916320717-main.pdf
title: Biofuel production from plant and algal biomass 
authors: Roman A. Voloshin, Margarita V. Rodionova, Sergey K. Zharmukhamedov, T. Nejat Veziroglu, Suleyman I. Allakhverdiev 
date: 2016
{'title': 'Biofuel production from plant and algal biomass ', 'authors': 'Roman A. Voloshin, Margarita V. Rodionova, Sergey K. Zharmukhamedov, T. Nejat Veziroglu, Suleyman I. Allakhverdiev ', 'date': '2016'}


1-s2.0-S0734975012001048-main.pdf
title: Advances in genetic engineering of marine algae
authors: 

title: Diversity of fungi isolated from carapace and gut of the marine crab Portunus sanguinolentus in northern waters of Taiwan
authors: Ami Shaumi, U-Cheang Cheng, Sheng-Yu Guo, E.B. Gareth Jones, Tin-Yam Chan and Ka-Lai Pang*
date: 2023
{'title': 'Diversity of fungi isolated from carapace and gut of the marine crab Portunus sanguinolentus in northern waters of Taiwan', 'authors': 'Ami Shaumi, U-Cheang Cheng, Sheng-Yu Guo, E.B. Gareth Jones, Tin-Yam Chan and Ka-Lai Pang*', 'date': '2023'}


10.1515_bot-2023-0002.pdf
title: Range expansion of some non-indigenous seaweeds along the coasts of Brittany – English Channel  
authors: Thomas Burel, Mathieu Helias, Michel Le Duﬀ  
date: 2023
{'title': 'Range expansion of some non-indigenous seaweeds along the coasts of Brittany – English Channel  ', 'authors': 'Thomas Burel, Mathieu Helias, Michel Le Duﬀ  ', 'date': '2023'}


10.1515_bot-2023-0009.pdf
title: Biodiversity of epiphytic marine macroalgae in Mexico: composition and current status

title: Algal Biofuels: Recent Advances and Future Prospects
authors: Sanjay Kumar Gupta, Anushree Malik, Faizal Bux (Editors)
date: 2019
{'title': 'Algal Biofuels: Recent Advances and Future Prospects', 'authors': 'Sanjay Kumar Gupta, Anushree Malik, Faizal Bux (Editors)', 'date': '2019'}


978-3-319-52666-9.pdf
title: Microbial Applications Vol.1: Bioremediation and Bioenergy
authors: Vipin Chandra Kalia, Prasun Kumar
date: 2017
{'title': 'Microbial Applications Vol.1: Bioremediation and Bioenergy', 'authors': 'Vipin Chandra Kalia, Prasun Kumar', 'date': '2017'}


978-3-319-68717-9.pdf
title: Medicinal Plants and Environmental Challenges
authors: Mansour Ghorbanpour, Ajit Varma
date: 2017
{'title': 'Medicinal Plants and Environmental Challenges', 'authors': 'Mansour Ghorbanpour, Ajit Varma', 'date': '2017'}


978-3-319-74703-3.pdf
title: Algae Biomass: Characteristics and Applications Towards Algae-based Products  
authors: Katarzyna Chojnacka, Piotr Paweł Wieczorek, Grzegorz Schroede

title: Violetonostoc minutum gen. et sp. nov. (Nostocales, Cyanobacteria) from a rocky substrate in China
authors: Fangfang Cai, Xin Peng, Renhui Li
date: 2020
{'title': 'Violetonostoc minutum gen. et sp. nov. (Nostocales, Cyanobacteria) from a rocky substrate in China', 'authors': 'Fangfang Cai, Xin Peng, Renhui Li', 'date': '2020'}


algae-2020-35-3-6.pdf
title: A new species of marine algae from Korea based on morphology and molecular data: Gelidium palmatum sp. nov. (Gelidiales, Rhodophyta)  
authors: Ga Hun Boo, Kyeong Mi Kim  
date: 2020
{'title': 'A new species of marine algae from Korea based on morphology and molecular data: Gelidium palmatum sp. nov. (Gelidiales, Rhodophyta)  ', 'authors': 'Ga Hun Boo, Kyeong Mi Kim  ', 'date': '2020'}


algae-2020-35-5-14.pdf
title: Protective effects of extracts from six local strains of Pyropia yezoensis against oxidative damage in vitro and in zebrafish model
authors: Yu-Lin Dai, Gwang Hoon Kim, Min-Cheol Kang, You-Jin Jeon
date: 2020
{'t

title: Revealing hidden diversity in the Sheathia arcuata morphospecies (Batrachospermales, Rhodophyta) including four new species
authors: Morgan L. Vis, Sunil Tiwari, Joshua R. Evans, Rosalina Stancheva, Robert G. Sheath, Bryan Kennedy, Janina Lee, Pertti Eloranta
date: 2020
{'title': 'Revealing hidden diversity in the Sheathia arcuata morphospecies (Batrachospermales, Rhodophyta) including four new species', 'authors': 'Morgan L. Vis, Sunil Tiwari, Joshua R. Evans, Rosalina Stancheva, Robert G. Sheath, Bryan Kennedy, Janina Lee, Pertti Eloranta', 'date': '2020'}


algae-2020-35-9-2.pdf
title: Semi-continuous cultivation of the mixotrophic dinoflagellate Gymnodinium smaydae, a new promising microalga for omega-3 production
authors: An Suk Lim, Hae Jin Jeong, Ji Hyun You, Sang Ah Park
date: 2020
{'title': 'Semi-continuous cultivation of the mixotrophic dinoflagellate Gymnodinium smaydae, a new promising microalga for omega-3 production', 'authors': 'An Suk Lim, Hae Jin Jeong, Ji Hyun 

...

title: Fucoidan Protects against Doxorubicin-Induced Cardiotoxicity by Reducing Oxidative Stress and Preventing Mitochondrial Function Injury
authors: Yuting Ji, Dekui Jin, Jingyi Qi, Xuan Wang, Chengying Zhang, Peng An, Yongting Luo, Junjie Luo
date: 2022
{'title': 'Fucoidan Protects against Doxorubicin-Induced Cardiotoxicity by Reducing Oxidative Stress and Preventing Mitochondrial Function Injury', 'authors': 'Yuting Ji, Dekui Jin, Jingyi Qi, Xuan Wang, Chengying Zhang, Peng An, Yongting Luo, Junjie Luo', 'date': '2022'}


PMC9505531.pdf
title: High Molecular Weight Fucoidan Restores Intestinal Integrity by Regulating Inflammation and Tight Junction Loss Induced by Methylglyoxal-Derived Hydroimidazolone-1  
authors: Jae-Min Lim, Hee Joon Yoo, Kwang-Won Lee  
date: 2022
{'title': 'High Molecular Weight Fucoidan Restores Intestinal Integrity by Regulating Inflammation and Tight Junction Loss Induced by Methylglyoxal-Derived Hydroimidazolone-1  ', 'authors': 'Jae-Min Lim, Hee Joon Yoo,

title: Impacts of agar gum and fucoidan on gel properties of surimi products without phosphate  
authors: Mingjing Zheng, Jinling Hong, Pengjie Chuai, Yanhong Chen, Hui Ni, Qingbiao Li, Zedong Jiang  
date: 2022
{'title': 'Impacts of agar gum and fucoidan on gel properties of surimi products without phosphate  ', 'authors': 'Mingjing Zheng, Jinling Hong, Pengjie Chuai, Yanhong Chen, Hui Ni, Qingbiao Li, Zedong Jiang  ', 'date': '2022'}


PMC9654794.pdf
title: Fucoidan Improves Growth, Digestive Tract Maturation, and Gut Microbiota in Large Yellow Croaker (Larimichthys crocea) Larvae  
authors: Zhaoyang Yin, Ye Gong, Yongtao Liu, Yuliang He, Chuanwei Yao, Wenxing Huang, Kangsen Mai, Qinghui Ai  
date: 2022
{'title': 'Fucoidan Improves Growth, Digestive Tract Maturation, and Gut Microbiota in Large Yellow Croaker (Larimichthys crocea) Larvae  ', 'authors': 'Zhaoyang Yin, Ye Gong, Yongtao Liu, Yuliang He, Chuanwei Yao, Wenxing Huang, Kangsen Mai, Qinghui Ai  ', 'date': '2022'}


PMC969163

Based on the context you've provided, it appears you might be referring to a general statement or an informational announcement from Elsevier regarding their COVID-19 Resource Centre rather than a specific document with identifiable authors and a publication date. Elsevier is a publisher and analytics company offering a variety of scientific, technical, and medical research papers, and the description here does not detail a single research article or document but rather describes Elsevier's overall initiative to provide COVID-19 resources and research for free.

However, if there are specific research articles or documents related to COVID-19 that you are seeking from the Elsevier COVID-19 Resource Centre, they would have their own individual titles, authors, and publication dates. Since the statement does not refer to a specific document, it's not possible to provide the title, authors, or publication date you're asking for.

If you need information on a specific document from the Els

title: Nano-Sized Fucoidan Interpolyelectrolyte Complexes: Recent Advances in Design and Prospects for Biomedical Applications  
authors: Natallia V. Dubashynskaya, Ekaterina R. Gasilova, and Yury A. Skorik  
date: 2023
{'title': 'Nano-Sized Fucoidan Interpolyelectrolyte Complexes: Recent Advances in Design and Prospects for Biomedical Applications  ', 'authors': 'Natallia V. Dubashynskaya, Ekaterina R. Gasilova, and Yury A. Skorik  ', 'date': '2023'}


PMC9917243.pdf
title: Interaction of High-Molecular Weight Fucoidan from Laminaria hyperborea with Natural Functions of the Retinal Pigment Epithelium  
authors: Philipp Dörschmann, Georg Kopplin, Johann Roider, Alexa Klettner  
date: 2023
{'title': 'Interaction of High-Molecular Weight Fucoidan from Laminaria hyperborea with Natural Functions of the Retinal Pigment Epithelium  ', 'authors': 'Philipp Dörschmann, Georg Kopplin, Johann Roider, Alexa Klettner  ', 'date': '2023'}


PMC9920843.pdf
title: Molecular Mechanism of Fucoidan Nanop

title: Removal and Utilization of Wastewater Nutrients for Algae Biomass and Biofuels  
authors: Erick W. Griffiths  
date: 2009
{'title': 'Removal and Utilization of Wastewater Nutrients for Algae Biomass and Biofuels  ', 'authors': 'Erick W. Griffiths  ', 'date': '2009'}


s11027-010-9255-9.pdf
title: Algae: the world’s most important “plants”—an introduction  
authors: Russell Leonard Chapman  
date: 2013  
{'title': 'Algae: the world’s most important “plants”—an introduction  ', 'authors': 'Russell Leonard Chapman  ', 'date': '2013'}


The Algae Worls.pdf
title: The Algae World
authors: Dinabandhu Sahoo, Joseph Seckbach
date: 2015
{'title': 'The Algae World', 'authors': 'Dinabandhu Sahoo, Joseph Seckbach', 'date': '2015'}


The potential of sustainable algal biofuel production using wastewater resources - ScienceDirect.pdf
title: The potential of sustainable algal biofuel production using wastewater resources
authors: Jon K. Pittman, Andrew P. Dean, Olumayowa Osundeko
date: 2011
{'

In [1]:
def write_strings_to_file(txt):
    with open("libraryAsText.txt", 'a', encoding='utf-8') as file:
            file.write(f"{txt}\n")


Then let's actually check out what's been loaded

In [27]:
#Unfold the list of pages
list_of_pages = [item for sublist in data for item in sublist]
print(type(list_of_pages))
print (f'There are {len(list_of_pages)} page(s) in the data')
print (f'There are {len(list_of_pages[0].page_content)} characters in a sample document')
print (f'A sample: {list_of_pages[3340].page_content[:10000]}')

<class 'list'>
There are 25363 page(s) in the data
There are 3954 characters in a sample document
A sample: power Ppv,ogenerated by the PV array is zero, the necessary power ( Pload) and
hydrogen ( mh2,user) to satisfy the end user demand are provided by the lead-acid
battery or fuel cell and hydrogen tank, and if Ppv,o>0 then the surplus energy is
used to charge the lead-acid accumulator, or in the electrolyser for the hydrogen
production. In detailed way, the optimal energy and hydrogen management strategycan be described in the following scenarios chronologically encountered in the
system operation:
1. In the case of P
pv,o¼0 and (1) if SoC b>SoC b,minthen the lead-acid battery and
hydrogen tank provide the required energy and hydrogen; (2) if
SoC b/C20SoC b,minthen the hydrogen tank provides the required energy to the
load through the fuel cell and (3) if SoC b/C20SoC b,minand SoC ht/C200 then for the
system safety the load and H 2user should be disconnected.
2. In the case of Ppv,

### Chunk data up into smaller documents
We can't attached a whole book as context to our LLM. It will recieve to much noise, and will likely not answer our question that well. Instead we divide the book into chunks. Then we will later only attach the really relevant chunks as context to the question we send til the llm. This will give us a sharper respons.

In [28]:
# Number of chunks we have
print (f'Now you have {len(list_of_pages)} chunks')

Now you have 25363 chunks


In [86]:
#lets see what is inside the chunks:
print(list_of_pages[14060])
texts = list_of_pages

page_content='Nutrients 2015, 7 245 \n \ngenerated by different sources. The immediate source of ammonia production is the purine nucleotide \ncycle [29], deamination of adenosine monophosphate to inosine monophosphate, and ammonia is the \nsubstantially elevated during intensive or prolonged exercise when the ATP utilization rate may exceed the production rate. The other source could be the gluconeogenesis process via deamination of several \namino acids produced by proteolysis. The direction of movement of ammonia or ammonium ion depends \non concentration and pH gradients between tissues, including the brain. Although e xercise -induced \nammonia toxicity is  transi ent and reversible relative to disease states, it  may affect continuing \ncoordinated activity in critical regions of the central nervous system. As shown  in the Figure 3B, s erum  \nammonia levels in the vehicle, FCD -1X and FCD -2X groups were 100  ± 5, 91 ± 9 and 83 ± 3 μmol/L, \nrespectively. Compared with the vehi

### Create embeddings for document chunks to get ready for semantic search

The technic for finding only the relevant chunks for our question is to do a semantic search. The way we prepare the cunks for this is through embedding them (getting a vector per document).

This will help us compare documents later on.
We use openAI's embeddingmodel for this.


In [90]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [91]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', 'YourAPIKey')
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

Let's test it out. I want to see which documents are most closely related to a query.



### Pinecone (a vectorstore in the cloud)


In [92]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY', 'YourAPIKey')
PINECONE_ENV = os.getenv('PINECONE_ENV', 'gcp-starter') 

from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY')
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

# configure client
pc = Pinecone(api_key=api_key)

In [93]:
from pinecone import ServerlessSpec, PodSpec

spec = PodSpec(environment=environment)

In [98]:
index_name = 'algaeopenai'
index = pc.Index(index_name)


In [99]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.25363,
 'namespaces': {'': {'vector_count': 25363}},
 'total_vector_count': 25363}

In [96]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(texts, embeddings, index_name=index_name)

### Query those docs to get your answer back

Great, those are just the docs which should hold our answer. Now we can pass those to a LangChain chain to query the LLM.

We could do this manually, but a chain is a convenient helper for us.

In [115]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

In [34]:
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [35]:
query = "In the context of hydrogen, what does THEUS stand for?"
docs = vectorstore.similarity_search(query, k=3)
docs[2]

Document(page_content='Chapter 31\nWater Is the Ultimate Source of Hydrogen\nEnergy: Scientiﬁc Citations and Quotations\nHussein K. Abdel-Aal and Nejat Veziroglu\nAbstract From the ancient times, mankind has always been aware that all life on\nearth depends upon water, the principal ingredient of all living cells. Its importancein forming the creation in all of its aspects in general and the living creatures in\nparticular, as well as human kind, animals, and plants, is evident to all of us. It is\nhighly important to know that water has played a major role for human kind. When\nthe Ionian philosopher Thales of Miletus (624–545 BC) replaced the Gods withNatural Laws as the force governing all phenomena; he made water the central\nelement in his theory.\nThis chapter addresses the issue that water, besides its indispensable usage by\nhuman beings in drinking and in everyday life, is the key element in providing lifewith energy, in the form of hydrogen .\nThe main aim of this work is to 

In [36]:
print('User:')
print(query)
print("Algaefessor:")
print(chain.run(input_documents=docs, question=query))
print(f"Source: {docs[2].metadata['source']} on page: {int(docs[2].metadata['page'])}")

User:
In the context of hydrogen, what does THEUS stand for?
Algaefessor:
THEUS stands for Totalized Hydrogen Energy Utilization System.
Source: Algae-Chatbot/978-3-319-17031-2.pdf on page: 449


In [37]:
query = "What is cyanobacteria and why is it classified as an algae"
docs = vectorstore.similarity_search(query, k=3)
print('User:')
print(query)
print("Algaefessor:")
print(chain.run(input_documents=docs, question=query))
print(f"Source: {docs[2].metadata['source']} on page: {int(docs[2].metadata['page'])}")

User:
What is cyanobacteria and why is it classified as an algae
Algaefessor:
Cyanobacteria, also known as blue-green algae, are a group of bacteria that have the ability to perform photosynthesis. They are classified as algae because they contain chlorophyll a and related compounds, which are characteristic of algae. Additionally, cyanobacteria share many structural features with bacteria, but their ability to convert atmospheric nitrogen into ammonia and their production of nitrogenous compounds and cyclic polyethers are more similar to algae.
Source: Algae-Chatbot/978-3-319-51010-1.pdf on page: 118
